In [1]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
pd.set_option('display.max_rows', 8)

!whoami

ihme\yongqx2


In [2]:
# /ihme/costeffectiveness/results/vivarium_csu_swissre_cancer/v4.1_family_history/swissre_coverage/2020_07_15_18_34_50/count_data
master_dir = 'C:/Users/yongqx2/Desktop/Sim Science/Swiss RE Cancer/'

In [3]:
fnames = ['person_time', 'transition_count']
df = {}
for fname in fnames:
    df[fname] = pd.read_csv(master_dir + 'count_data_model_4/' + fname + '.csv').iloc[:, 1:]
    df[fname]['risk'] = df[fname]['age_cohort'].map(lambda x: 'family_history_' + x.split('_family_history_')[1])
    df[fname]['age_cohort'] = df[fname]['age_cohort'].map(lambda x: x.split('_family_history_')[0])

In [4]:
pt = (df['person_time']
      .groupby(['input_draw', 'scenario', 'risk'])
      .value.sum()
      .reset_index())
pt

,input_draw,scenario,risk,value
0,3,alternative,family_history_negative,1.449312e+08
1,3,alternative,family_history_positive,2.949030e+06
2,3,baseline,family_history_negative,1.449312e+08
3,3,baseline,family_history_positive,2.949030e+06
...,...,...,...,...
396,990,alternative,family_history_negative,1.448990e+08
397,990,alternative,family_history_positive,2.968630e+06
398,990,baseline,family_history_negative,1.448990e+08
399,990,baseline,family_history_positive,2.968630e+06


In [5]:
prop_fh = (pt
           .loc[pt.risk == 'family_history_positive']
           .set_index(['input_draw', 'scenario']).value
           .div(pt.groupby(['input_draw', 'scenario']).value.sum())
           .reset_index())

prop_fh_summary = (prop_fh
                   .groupby('scenario').value
                   .describe(percentiles=[.025, .975])
                   .filter(['mean', '2.5%', '97.5%'])
                   .reset_index())
prop_fh_summary

,scenario,mean,2.5%,97.5%
0,alternative,0.02,0.019916,0.020106
1,baseline,0.02,0.019916,0.020106


#### The prevalence of family history in both scenario = 0.02 (95%CI 0.019916-0.020106)

In [6]:
def calc_relative_risk(transition: pd.DataFrame, pt: pd.DataFrame, measure: str):
    count = (transition
             .loc[transition.measure == measure]
             .groupby(['input_draw', 'scenario', 'measure', 'risk'])
             .value.sum()
             .reset_index())
    rate = (count
            .set_index(['input_draw', 'scenario', 'measure', 'risk'])
            .div(pt.set_index(['input_draw', 'scenario', 'risk']))
            .reset_index())
    
    pos = (rate
           .loc[rate.risk == 'family_history_positive']
           .set_index(['input_draw', 'scenario', 'measure'])
           .value)
    neg = (rate
           .loc[rate.risk == 'family_history_negative']
           .set_index(['input_draw', 'scenario', 'measure'])
           .value)
    
    # rr_fh = incidence_rate_among_fh_positive / incidence_rate_among_fh_negative
    rr = pos.div(neg).reset_index()
    rr_summary = (rr
                  .groupby(['scenario', 'measure']).value
                  .describe(percentiles=[.025, .975])
                  .filter(['mean', '2.5%', '97.5%'])
                  .reset_index())
    return rr_summary

In [7]:
measures = [
    'susceptible_to_breast_cancer_to_lobular_carcinoma_in_situ_event_count',
    'susceptible_to_breast_cancer_to_ductal_carcinoma_in_situ_event_count',
    'ductal_carcinoma_in_situ_to_breast_cancer_event_count',
    'lobular_carcinoma_in_situ_to_breast_cancer_event_count',
]
relative_risk = pd.DataFrame()
for measure in measures:
    rr = calc_relative_risk(df['transition_count'], pt, measure)
    relative_risk = relative_risk.append(rr, ignore_index=True)

In [8]:
relative_risk

,scenario,measure,mean,2.5%,97.5%
0,alternative,susceptible_to_breast_cancer_to_lobular_carcin...,1.878213,1.512736,2.159580
1,baseline,susceptible_to_breast_cancer_to_lobular_carcin...,1.878213,1.512736,2.159580
2,alternative,susceptible_to_breast_cancer_to_ductal_carcino...,1.881029,1.806731,1.959778
3,baseline,susceptible_to_breast_cancer_to_ductal_carcino...,1.881029,1.806731,1.959778
4,alternative,ductal_carcinoma_in_situ_to_breast_cancer_even...,1.762125,1.599004,1.866133
5,baseline,ductal_carcinoma_in_situ_to_breast_cancer_even...,1.762125,1.599004,1.866133
6,alternative,lobular_carcinoma_in_situ_to_breast_cancer_eve...,1.670813,1.148208,1.937879
7,baseline,lobular_carcinoma_in_situ_to_breast_cancer_eve...,1.670813,1.148208,1.937879


### The relative risk of family history for `S to DCIS` and `S to LCIS` are close to 1.9, but we expect the relative risk of family history for `DCIS to Breast Cancer` and `LCIS to Breast Cancer` equal to 1.